<a href="https://colab.research.google.com/github/hyrule-coder/langchain-book-learning/blob/main/chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Advanced RAG

## 6.1 Advanced RAGの概要

## 6.2 ハンズオンの準備

In [ ]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
     langchain-community==0.3.0 GitPython==3.1.43 \
     langchain-chroma==0.1.4 tavily-python==0.5.0

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "agent-book"
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [ ]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

390


In [ ]:
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

<ipython-input-4-a8ae901c0614>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈:"""
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini",temperature=0)

retriever = db.as_retriever()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、ユーザーは自分のニーズに応じてさまざまなコンポーネントを選択して使用できます。'

## 6.3 検索クエリの工夫

### HyDE (Hypothetical Document Embeddings)

In [ ]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発、運用、デプロイの各段階を簡素化することを目的としています。\n\n主な特徴は以下の通りです：\n\n1. **標準化されたコンポーネントインターフェース**: LangChainは、AIアプリケーションに必要な主要コンポーネントのための共通インターフェースを提供し、異なるプロバイダー間での切り替えを容易にします。\n\n2. **オーケストレーション**: 複数のコンポーネントやモデルを組み合わせて複雑なアプリケーションを構築するためのオーケストレーション機能を提供します。これにより、複雑な制御フローや人間の介入を必要とするアプリケーションを構築できます。\n\n3. **可観測性と評価**: LangChainは、アプリケーションの挙動を監視し、迅速に評価するためのツールを提供します。これにより、開発者はアプリケーションのパフォーマンスを最適化しやすくなります。\n\nLangChainは、さまざまなプロバイダーとの統合をサポートし、開発者が自分のニーズに合わせてコンポーネントを選択できる柔軟性を持っています。また、LangGraphというライブラリを使用することで、コンポーネントを組み合わせて生産準備が整ったアプリケーションを構築することができます。さらに、LangSmithを利用することで、アプリケーションのトレースや評価を行い、開発プロセスを改善することが可能です。'

### 複数の検索クエリの生成

In [ ]:
from pydantic import BaseModel, Field

class QueryGenerationOutput(BaseModel):
  queries: list[str] = Field(..., description="検索クエリのリスト")

query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
３つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    |(lambda x: x.queries)
)

multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChainの概要を教えて")


'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。これにより、開発者は異なるプロバイダー間での切り替えが容易になります。\n\nまた、LangChainは、複雑なアプリケーションの構築を支援するためのオーケストレーション機能や、アプリケーションの可視性と評価を向上させるための機能も提供しています。これにより、開発者はアプリケーションの動作を理解しやすくなり、迅速に問題を特定して解決することが可能になります。'

## 6.4 検索の工夫

### RAG-Fusion

In [ ]:
from langchain_core.documents import Document

def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60
)->list[str]:
  content_score_mapping={}

  for docs in retriever_outputs:
    for rank, doc in enumerate(docs):
      content = doc.page_content

      if content not in content_score_mapping:
        content_score_mapping[content] = 0

      content_score_mapping[content] += 1 / (rank + k)

  ranked = sorted(
      content_score_mapping.items(),
      key=lambda x: x[1],
      reverse=True,
  )
  return [content for content, _ in ranked]


In [ ]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、アプリケーションのライフサイクルの各段階を簡素化することを目的としています。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換することができます。\n\nLangChainは、さまざまなプロバイダーと統合できる標準インターフェースを実装しており、開発者が異なるコンポーネントを簡単に組み合わせて使用できるようにしています。また、複雑なアプリケーションのオーケストレーションをサポートするために、LangGraphというライブラリを提供しています。これにより、エージェントやマルチエージェントアプリケーションの構築が容易になります。\n\nさらに、LangSmithを使用することで、アプリケーションのトレースや評価が可能になり、開発者は自信を持ってアプリケーションを改善できます。全体として、LangChainはAIアプリケーションの開発を効率化し、開発者が迅速に価値を提供できるように設計されています。'

### Cohereのりランクモデルを使用する準備

In [ ]:
!pip install langchain-cohere==0.3.0
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.1 MB/s eta 0:00:00


### Cohereのりランクモデルの導入

In [ ]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document

def rerank(inp: dict[str, Any], top_n: int = 3)->list[Document]:
  question = inp["question"]
  documents = inp["documents"]

  cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
  return cohere_reranker.compress_documents(documents=documents, query=question)

rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストア）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、開発者が特定のニーズに応じてコンポーネントを選択して使用できる柔軟性を提供します。'

## 6.5 複数のRetrieversを使う工夫

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [ ]:
from enum import Enum

class Route(str, Enum):
  langchain_document = "langchain_document"
  web = "web"

class RouteOutput(BaseModel):
  route: Route

route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [ ]:
def routed_retriever(inp: dict[str, Any])->list[Document]:
  question = inp["question"]
  route = inp["route"]

  if route == Route.langchain_document:
    return langchain_document_retriever.get_relevant_documents(question)
  elif route == Route.web:
    return web_retriever.get_relevant_documents(question)

  raise ValueError(f"Unknown retriever: {route}")

route_rag_chain = (
  {
    "question": RunnablePassthrough(),
    "route": route_chain,
  }
  | RunnablePassthrough.assign(context=routed_retriever)
  | prompt | model | StrOutputParser()
)

In [ ]:
route_rag_chain.invoke("LangChainの概要を教えて")

<ipython-input-21-053bad158836>:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  return langchain_document_retriever.get_relevant_documents(question)


'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストア）に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、特に`langchain-core`、統合パッケージ、`langchain`、`langchain-community`、`langgraph`などが含まれています。\n\nさらに、LangChainは、アプリケーションの複雑さに応じて、LCEL（LangChain Expression Language）を使用してランナブルを組み合わせることができ、効率的なオーケストレーションを実現します。'

In [ ]:
route_rag_chain.invoke("東京の今日の天気は？")

'東京の今日、11月19日(火)の天気は「晴時々曇」で、最高気温は13℃、最低気温は8℃です。降水確率は0％で、風は北の風から北東の風に変わります。'

### ハイブリッド検索の実装

In [ ]:
!pip install rank-bm25==0.2.2

In [ ]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config({"run_name": "chroma_retriever"})

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [ ]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [ ]:
hybrid_rag_chain=(
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります：\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、PythonとJavaScriptの両方のライブラリが提供されています。'